# Figure3

## Figure 3

Hierarchical clustering heatmap of FDR-Significant lipid compounds

## Load necessary libraries

In [ ]:
library(readxl)
library(tidyr)
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: grid

ComplexHeatmap version 2.18.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))

## Function to perform FDR adjustment for a *single Molecule List*

In [ ]:
perform_fdr <- function(data) {
  data %>%
    mutate(
      fdr_adjusted_p = p.adjust(p.value, method = "BH"),
      significant = fdr_adjusted_p < 0.05
    )
}

# Perform FDR adjustment for each Molecule List separately
fdr_results <- anova_results %>%
  group_by(`Molecule List`) %>%
  group_modify(~perform_fdr(.x)) %>%
  ungroup()


# Filter for significant compounds after FDR correction
significant_compounds_fdr <- fdr_results %>%
  filter(significant == TRUE) %>%
  select(GlobalID, `Molecule List`, term) %>%
  distinct()

# Determine the effect for each significant compound
compound_effects_fdr <- significant_compounds_fdr %>%
  group_by(GlobalID) %>%
  summarise(
    Effect = case_when(
      any(term == "Type1") ~ "Climate",
      any(term == "Type2") ~ "Water",
      any(term == "Type1:Type2") ~ "Interaction",
      TRUE ~ "Unknown"
    )
  )

# Filter the combined data to include only the significant compounds after FDR
significant_data_fdr <- combined_df %>%
  filter(GlobalID %in% significant_compounds_fdr$GlobalID)

## Create a pivot table for the heatmap

In [ ]:
heatmap_data_fdr <- significant_data_fdr %>%
  group_by(GlobalID, Molecule, `Molecule List`, Treatment) %>%
  summarise(Measurement = mean(Measurement, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(names_from = Treatment, values_from = Measurement) %>%
  left_join(compound_effects_fdr, by = "GlobalID")

# Prepare the matrix for heatmap
heatmap_data_fdr <- heatmap_data_fdr %>%
  mutate(RowLabel = paste(Molecule, " (", GlobalID, ")", ", ", `Molecule List`, sep = ""))

rownames(heatmap_data_fdr) <- heatmap_data_fdr$RowLabel

quartz_off_screen 
                2 

[1] "FDR-corrected heatmap with compound names and bottom legends has been saved as 'complex_heatmap_fdr_significant_compounds_with_names.pdf'"